In [9]:
# Importing the necessary libraries to be used for this project
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import telegram
from datetime import datetime, timedelta

##  Loading df
df = pd.read_parquet('blood_donation_retention_2024.parquet', engine = 'auto')
df = df[['donor_id', 'visit_date']]

df['year'] = df['visit_date'].apply(lambda x: x.split('-')[0])
df['last_visit'] = np.where(df['donor_id'].shift() == df['donor_id'], df['visit_date'].shift(), np.nan)

df['visit_date'] = pd.to_datetime(df['visit_date'])
df['last_visit'] = pd.to_datetime(df['last_visit'])

df['difference'] = df['visit_date'] - df['last_visit']
df['difference'] = df['difference'].dt.days

df['donation_new'] = np.where(df['donor_id'].eq(df['donor_id'].shift()), 0, 1)
df['donation_regular'] = np.where(df['difference'] <= 730, 1, 0)
df['donation_irregular'] = np.where(df['difference'] > 730, 1, 0)

display(df.head(20))
display(df.tail(20))


,donor_id,visit_date,year,last_visit,difference,donation_new,donation_regular,donation_irregular
0,00000,2013-11-03,2013,NaT,NaN,1,0,0
1,00000,2014-05-26,2014,2013-11-03,204.0,0,1,0
2,00000,2015-01-25,2015,2014-05-26,244.0,0,1,0
3,00000,2015-08-09,2015,2015-01-25,196.0,0,1,0
4,00000,2016-03-10,2016,2015-08-09,214.0,0,1,0
5,00000,2016-08-14,2016,2016-03-10,157.0,0,1,0
6,00000,2016-12-19,2016,2016-08-14,127.0,0,1,0
7,00000,2017-04-20,2017,2016-12-19,122.0,0,1,0
8,00000,2018-03-19,2018,2017-04-20,333.0,0,1,0
9,00000,2018-12-31,2018,2018-03-19,287.0,0,1,0


,donor_id,visit_date,year,last_visit,difference,donation_new,donation_regular,donation_irregular
6257088,01Wxs,2024-01-30,2024,NaT,NaN,1,0,0
6257089,02kUu,2024-01-30,2024,NaT,NaN,1,0,0
6257090,00kl0,2024-01-30,2024,NaT,NaN,1,0,0
6257091,09qDd,2024-01-30,2024,NaT,NaN,1,0,0
6257092,03BhD,2024-01-30,2024,NaT,NaN,1,0,0
6257093,01dwa,2024-01-30,2024,NaT,NaN,1,0,0
6257094,04f3y,2024-01-30,2024,NaT,NaN,1,0,0
6257095,00FxG,2024-01-30,2024,NaT,NaN,1,0,0
6257096,00kmm,2024-01-30,2024,NaT,NaN,1,0,0
6257097,01OYx,2024-01-30,2024,NaT,NaN,1,0,0


In [7]:
df = df[['year', 'donation_new', 'donation_regular', 'donation_irregular']]
df = df.groupby('year').sum().reset_index()
df['donation_total'] = df['donation_new'] + df['donation_regular'] + df['donation_irregular']

display(df)

,year,donation_new,donation_regular,donation_irregular,donation_total
0,2012,334794,89932,0,424726
1,2013,229992,209915,0,439907
2,2014,227633,246595,13578,487806
3,2015,220147,282272,33403,535822
4,2016,196831,290251,46207,533289
5,2017,178624,293242,57258,529124
6,2018,182644,311325,71135,565104
7,2019,173828,311594,75355,560777
8,2020,128752,294234,74604,497590
9,2021,106520,321472,71773,499765
